In [26]:
import pandas as pd
import yaml
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

In [27]:
import os

os.getcwd()

'e:\\Videos\\My courses\\DEPI\\Final Project'

In [35]:
os.chdir('Final Project')
os.getcwd()


'e:\\Videos\\My courses\\DEPI\\Final Project'

In [36]:
with open('configs/paths.yaml','r') as f:
    config = yaml.safe_load(f)

config


{'dir': {'data_dir': 'data',
  'artifacts_dir': 'artifacts',
  'raw_dir': 'data/raw',
  'processed_dir': 'data/processed',
  'models_dir': 'models',
  'config_dir': 'configs'},
 'paths': {'raw_data_path': 'data/raw/Superstore.csv',
  'processed_data_path': 'data/processed/processed_data.csv',
  'transformer': 'models/transformer.pkl'}}

In [69]:
data_path = config['paths']['processed_data_path']
df = pd.read_csv(data_path,encoding='ISO-8859-1')

df.head()

,Order Date,Ship Mode,Segment,City,State,Region,Category,Sub-Category,Sales,Quantity,...,price_per_unit,month,year,day_of_week,week_of_year,sales_lag_1,sales_lag_2,sales_lag_3,sales_lag_4,sales_lag_5
0,2011-06-09,Standard Class,Consumer,Los Angeles,California,West,Furniture,Furnishings,48.860,7,...,6.980,6,2011,3,23,22.368,957.5775,14.6200,731.9400,261.9600
1,2011-06-09,Standard Class,Consumer,Los Angeles,California,West,Office Supplies,Art,7.280,4,...,1.820,6,2011,3,23,48.860,22.3680,957.5775,14.6200,731.9400
2,2011-06-09,Standard Class,Consumer,Los Angeles,California,West,Technology,Phones,907.152,6,...,151.192,6,2011,3,23,7.280,48.8600,22.3680,957.5775,14.6200
3,2011-06-09,Standard Class,Consumer,Los Angeles,California,West,Office Supplies,Binders,18.504,3,...,6.168,6,2011,3,23,907.152,7.2800,48.8600,22.3680,957.5775
4,2011-06-09,Standard Class,Consumer,Los Angeles,California,West,Office Supplies,Appliances,114.900,5,...,22.980,6,2011,3,23,18.504,907.1520,7.2800,48.8600,22.3680


In [70]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9989 entries, 0 to 9988
Data columns (total 23 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Order Date      9989 non-null   object 
 1   Ship Mode       9989 non-null   object 
 2   Segment         9989 non-null   object 
 3   City            9989 non-null   object 
 4   State           9989 non-null   object 
 5   Region          9989 non-null   object 
 6   Category        9989 non-null   object 
 7   Sub-Category    9989 non-null   object 
 8   Sales           9989 non-null   float64
 9   Quantity        9989 non-null   int64  
 10  Discount        9989 non-null   float64
 11  Profit          9989 non-null   float64
 12  Time_taken      9989 non-null   int64  
 13  price_per_unit  9989 non-null   float64
 14  month           9989 non-null   int64  
 15  year            9989 non-null   int64  
 16  day_of_week     9989 non-null   int64  
 17  week_of_year    9989 non-null   i

# prepare the data 

In [67]:
from sklearn.preprocessing import QuantileTransformer, StandardScaler,LabelEncoder

In [71]:
df['Order Date'] = pd.to_datetime(df['Order Date'])

train = df[df['year'].between(2011, 2013)]
test = df[df['year'].between(2014, 2014)]

dates_train = train['Order Date']
dates_test = test['Order Date']


x_train = train.drop(['Sales','Order Date'],axis=1)
x_test = test.drop(['Sales','Order Date'],axis=1)
y_train = train['Sales']
y_test = test['Sales']


def preprocess_data(data):
    """
    Preprocess the dataset by applying cyclical encoding, scaling, one-hot encoding, and label encoding.

    Parameters:
        data (pd.DataFrame): The input dataset to preprocess.

    Returns:
        pd.DataFrame: The preprocessed dataset.
    """


    # Step 2: Scaling Numerical Columns
    # Quantile Scaling for skewed columns
    quantile_columns = ['Profit', 'price_per_unit']
    quantile_transformer = QuantileTransformer(output_distribution='normal', random_state=42)
    data[quantile_columns] = quantile_transformer.fit_transform(data[quantile_columns])

    # Standard Scaling for normally distributed numerical columns
    num_scaling = ['Quantity', 'Discount', 'Time_taken']
    scaler = StandardScaler()
    data[num_scaling] = scaler.fit_transform(data[num_scaling])

    col_label = data.select_dtypes('object').columns.tolist()
    label_encoder = LabelEncoder()
    for col in col_label:
        data[col] = label_encoder.fit_transform(data[col])

    return data



In [72]:
# dates_train,dates_test,x_train,x_test,y_train,y_test = apply_pipeline(data_processed,transformer)

x_train = preprocess_data(x_train)
x_test = preprocess_data(x_test)

In [63]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6670 entries, 0 to 6669
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Profit          6670 non-null   float64
 1   price_per_unit  6670 non-null   float64
 2   Quantity        6670 non-null   float64
 3   Discount        6670 non-null   float64
 4   Time_taken      6670 non-null   float64
 5   Ship Mode       6670 non-null   float64
 6   Segment         6670 non-null   float64
 7   City            6670 non-null   float64
 8   State           6670 non-null   float64
 9   Region          6670 non-null   float64
 10  Category        6670 non-null   float64
 11  Sub-Category    6670 non-null   float64
 12  month           6670 non-null   float64
 13  year            6670 non-null   float64
 14  day_of_week     6670 non-null   float64
 15  week_of_year    6670 non-null   float64
 16  sales_lag_1     6670 non-null   float64
 17  sales_lag_2     6670 non-null   f

In [42]:
print('x_train shape: ',x_train.shape)
print('x_test shape: ',x_test.shape)
print('y_train shape: ',y_train.shape)
print('y_test shape: ',y_test.shape)
print('dates_train shape: ',dates_train.shape)
print('dates_test shape: ',dates_test.shape)


x_train shape:  (6670, 21)
x_test shape:  (3319, 21)
y_train shape:  (6670,)
y_test shape:  (3319,)
dates_train shape:  (6670,)
dates_test shape:  (3319,)


In [73]:
x_train.info()
    

<class 'pandas.core.frame.DataFrame'>
Index: 6670 entries, 0 to 9984
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Ship Mode       6670 non-null   int32  
 1   Segment         6670 non-null   int32  
 2   City            6670 non-null   int32  
 3   State           6670 non-null   int32  
 4   Region          6670 non-null   int32  
 5   Category        6670 non-null   int32  
 6   Sub-Category    6670 non-null   int32  
 7   Quantity        6670 non-null   float64
 8   Discount        6670 non-null   float64
 9   Profit          6670 non-null   float64
 10  Time_taken      6670 non-null   float64
 11  price_per_unit  6670 non-null   float64
 12  month           6670 non-null   int64  
 13  year            6670 non-null   int64  
 14  day_of_week     6670 non-null   int64  
 15  week_of_year    6670 non-null   int64  
 16  sales_lag_1     6670 non-null   float64
 17  sales_lag_2     6670 non-null   float6

# XGBoost

In [44]:
import xgboost as xgb
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.metrics import mean_squared_error, r2_score
import optuna
import mlflow


In [86]:
def plot_importance_features(model,booster):
    '''
  """
  Plots feature importance for an XGBoost model.

  Args:
  - model: A trained XGBoost model

    - booster: The type of booster to use. Can be 'gbtree' or 'gblinear'.
    '''
    
    fig,ax = plt.subplots(figsize=(12,6))

    xgb.plot_importance(model,
                    ax=ax)
    plt.title(f'Feature Importance ({booster})')
    plt.xlabel('Weight')
    plt.ylabel('Features')
    
    plt.tight_layout()
    plt.close()
    return fig


In [46]:
def get_or_create_experiment(experiment_name):
  """
  Retrieve the ID of an existing MLflow experiment or create a new one if it doesn't exist.

  This function checks if an experiment with the given name exists within MLflow.
  If it does, the function returns its ID. If not, it creates a new experiment
  with the provided name and returns its ID.

  Parameters:
  - experiment_name (str): Name of the MLflow experiment.

  Returns:
  - str: ID of the existing or newly created MLflow experiment.
  """

  if experiment := mlflow.get_experiment_by_name(experiment_name):
      return experiment.experiment_id
  else:
      return mlflow.create_experiment(experiment_name)

In [47]:
def plot_forecast(dates_test,y_test,y_pred):

    fig , ax = plt.subplots(figsize=(15,6))
    # Convert y_test and y_pred to pandas Series
    dates_test = pd.to_datetime(dates_test)

    y_test_series = pd.Series(y_test.values, index=dates_test)
    y_pred_series = pd.Series(y_pred, index=dates_test)

    y_test_resampled = y_test_series.resample('W').sum()
    y_pred_resampled = y_pred_series.resample('W').sum()

    ax.plot(y_test_resampled.index, y_test_resampled.values, label='Actual (Monthly)')
    ax.plot(y_pred_resampled.index, y_pred_resampled.values, label='XGBoost Forecast (Weekly)', linestyle='--')

    ax.set_xlabel('Date')
    ax.set_ylabel('Sales')
    ax.set_title('XGBoost Forecast vs Actual Sales (Optuna)')
    ax.legend()
    plt.close()
    return fig
    


In [74]:
dtrain = xgb.DMatrix(x_train,label=y_train)
dtest = xgb.DMatrix(x_test,label=y_test)

## XGBoost Optuna

In [75]:
def objective(trial):

      # Define hyperparameters
    params = {
          "objective": "reg:squarederror",
          "eval_metric": "rmse",
          "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
          "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
          "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
      }

    if params["booster"] == "gbtree" or params["booster"] == "dart":
          params["max_depth"] = trial.suggest_int("max_depth", 1, 9)
          params["eta"] = trial.suggest_float("eta", 1e-8, 1.0,)
          params["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0,)
          params["grow_policy"] = trial.suggest_categorical(
              "grow_policy", ["depthwise", "lossguide"]
          )

      # Train XGBoost model
    bst = xgb.train(params, dtrain)
    preds = bst.predict(dtest)
    error = mean_squared_error(y_test, preds)

    return error

In [76]:
study = optuna.create_study(direction='minimize')
study.optimize(objective,n_trials=500,show_progress_bar=True)

[I 2025-05-15 08:21:27,981] A new study created in memory with name: no-name-434a2a1b-9237-418a-86b6-d5e3d7ce4bec


  0%|          | 0/500 [00:00<?, ?it/s]

[I 2025-05-15 08:21:28,024] Trial 0 finished with value: 213551.12391161063 and parameters: {'booster': 'gblinear', 'lambda': 1.2045875527386668e-07, 'alpha': 2.9547288221158963e-07}. Best is trial 0 with value: 213551.12391161063.
[I 2025-05-15 08:21:28,085] Trial 1 finished with value: 40889.59566153668 and parameters: {'booster': 'gbtree', 'lambda': 0.21935745663066103, 'alpha': 0.008683880478308892, 'max_depth': 7, 'eta': 0.6567441753741253, 'gamma': 0.4850603499178953, 'grow_policy': 'depthwise'}. Best is trial 1 with value: 40889.59566153668.
[I 2025-05-15 08:21:28,104] Trial 2 finished with value: 93202.98632455125 and parameters: {'booster': 'gbtree', 'lambda': 0.1180962758799237, 'alpha': 0.26824513769022623, 'max_depth': 1, 'eta': 0.36005560307385415, 'gamma': 0.8963237755758748, 'grow_policy': 'lossguide'}. Best is trial 1 with value: 40889.59566153668.
[I 2025-05-15 08:21:28,339] Trial 3 finished with value: 292894.0854785175 and parameters: {'booster': 'dart', 'lambda': 9.

In [81]:
experiment_id = get_or_create_experiment(experiment_name='sales_forecast')
experiment_id

'750350709773541021'

In [91]:
mlflow.set_experiment(experiment_name='sales_forecast')
with mlflow.start_run(run_name='XGBoost'):
    mlflow.log_params(study.best_params)
    best_params = study.best_params

    best_model = xgb.train(best_params,dtrain)

    mlflow.xgboost.log_model(xgb_model=best_model,
                             artifact_path='models',
                             registered_model_name='xgb_model')

    y_pred = best_model.predict(dtest)

    mse_xgb = mean_squared_error(y_test,y_pred)
    rmse_xgb = np.sqrt(mse_xgb)
    r2_xgb = r2_score(y_test,y_pred)

    mlflow.log_metrics({'mse':mse_xgb,'rmse':rmse_xgb,'r2':r2_xgb})

    mlflow.set_tags({
        'model_type':'XGBoost',
        'experiment_id':experiment_id,
        'run_name':'XGBoost',
        'run_id':mlflow.active_run().info.run_id
    })

    fig_importance = plot_importance_features(best_model,booster='dart')
    mlflow.log_figure(fig_importance,artifact_file="feature_importances_xgb.png")

    fig_forecast = plot_forecast(dates_test,y_test,y_pred)
    mlflow.log_figure(fig_forecast,artifact_file="forecast_xgb.png")

    print('mse for xgb: ',mse_xgb)
    print('rmse for xgb: ',rmse_xgb)
    print('r2 for xgb: ',r2_xgb)

    mlflow.end_run()



c:\Users\Check-In\anaconda3\Lib\site-packages\mlflow\xgboost\__init__.py:168: UserWarning: [08:39:20] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/05/15 08:39:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'xgb_model' already exists. Creating a new version of this model...
Created version '1' of model 'xgb_model'.


mse for xgb:  23803.271906872444
rmse for xgb:  154.28309015207222
r2 for xgb:  0.9303489495897066


In [78]:
print('Best trial:')
print(study.best_trial)

Best trial:
FrozenTrial(number=417, state=1, values=[23803.271906872444], datetime_start=datetime.datetime(2025, 5, 15, 8, 22, 12, 349393), datetime_complete=datetime.datetime(2025, 5, 15, 8, 22, 12, 460394), params={'booster': 'dart', 'lambda': 8.435739953508957e-05, 'alpha': 4.9540633901605e-06, 'max_depth': 4, 'eta': 0.29452292540598635, 'gamma': 0.19633522750225307, 'grow_policy': 'lossguide'}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'booster': CategoricalDistribution(choices=('gbtree', 'gblinear', 'dart')), 'lambda': FloatDistribution(high=1.0, log=True, low=1e-08, step=None), 'alpha': FloatDistribution(high=1.0, log=True, low=1e-08, step=None), 'max_depth': IntDistribution(high=9, log=False, low=1, step=1), 'eta': FloatDistribution(high=1.0, log=False, low=1e-08, step=None), 'gamma': FloatDistribution(high=1.0, log=False, low=1e-08, step=None), 'grow_policy': CategoricalDistribution(choices=('depthwise', 'lossguide'))}, trial_id=417, value=None)


# Random Forest 

In [92]:
from sklearn.ensemble import RandomForestRegressor
# Define objective function
def objective(trial):
    # Suggest values for hyperparameters
    params = {
    'n_estimators' : trial.suggest_int("n_estimators", 10, 200, log=True),
    'max_depth' : trial.suggest_int("max_depth", 2, 32),
    'min_samples_split' : trial.suggest_int("min_samples_split", 2, 10),
    'min_samples_leaf' :trial.suggest_int("min_samples_leaf", 1, 10)
    }
    # Create and fit random forest model
    model = RandomForestRegressor(**params)
    model.fit(x_train, y_train)

    # Make predictions and calculate RMSE
    y_pred = model.predict(x_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    r2 = r2_score(y_test, y_pred)

    # Return MAE
    return rmse

study = optuna.create_study(direction='minimize')

study.optimize(objective,n_trials=20,show_progress_bar=True)

[I 2025-05-15 08:45:52,915] A new study created in memory with name: no-name-c614e0f6-ebc3-4ee0-876b-30ea8d08e396


  0%|          | 0/20 [00:00<?, ?it/s]

[I 2025-05-15 08:46:03,435] Trial 0 finished with value: 123.0777409744588 and parameters: {'n_estimators': 144, 'max_depth': 20, 'min_samples_split': 6, 'min_samples_leaf': 1}. Best is trial 0 with value: 123.0777409744588.
[I 2025-05-15 08:46:06,556] Trial 1 finished with value: 208.99884063494216 and parameters: {'n_estimators': 61, 'max_depth': 9, 'min_samples_split': 3, 'min_samples_leaf': 10}. Best is trial 0 with value: 123.0777409744588.
[I 2025-05-15 08:46:07,497] Trial 2 finished with value: 141.215466236946 and parameters: {'n_estimators': 13, 'max_depth': 18, 'min_samples_split': 3, 'min_samples_leaf': 7}. Best is trial 0 with value: 123.0777409744588.
[I 2025-05-15 08:46:08,807] Trial 3 finished with value: 200.5491573834895 and parameters: {'n_estimators': 22, 'max_depth': 30, 'min_samples_split': 3, 'min_samples_leaf': 10}. Best is trial 0 with value: 123.0777409744588.
[I 2025-05-15 08:46:09,078] Trial 4 finished with value: 270.729233739104 and parameters: {'n_estimato

In [93]:
mlflow.set_experiment(experiment_name='sales_forecast')
with mlflow.start_run(run_name='Random Forest'):
    mlflow.log_params(study.best_params)
    best_params = study.best_params

    best_model = RandomForestRegressor(**best_params)
    best_model.fit(x_train,y_train)

    mlflow.sklearn.log_model(sk_model=best_model,
                             artifact_path='models',
                             registered_model_name='rf_model')
    
    y_pred = best_model.predict(x_test)

    mse_rf = mean_squared_error(y_test,y_pred)
    rmse_rf = np.sqrt(mse_rf)
    r2_rf = r2_score(y_test,y_pred)

    mlflow.log_metrics({'mse':mse_rf,'rmse':rmse_rf,'r2':r2_rf})

    mlflow.set_tags({
        'model_type':'Random Forest',
        'experiment_id':experiment_id,
        'run_name':'Random Forest',
        'run_id':mlflow.active_run().info.run_id
    })


    fig_forecast = plot_forecast(dates_test,y_test,y_pred)
    mlflow.log_figure(fig_forecast,artifact_file="forecast_rf.png")

    print('mse for rf: ',mse_rf)
    print('rmse for rf: ',rmse_rf)
    print('r2 for rf: ',r2_rf)

    mlflow.end_run()
    

2025/05/15 08:49:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'rf_model'.
Created version '1' of model 'rf_model'.


mse for rf:  13424.5362558515
rmse for rf:  115.86430104156975
r2 for rf:  0.960718297251347
